# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [3]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [4]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.031523811685561, 10.0)

In [5]:
# Put your code here to price the corporate bond

def corp_bond(mat=10, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000):
    U = np.random.uniform(size=n_sample) ### <-- YOUR ANSWER HERE
    default_time = -(1/def_rate)*np.log(U) ### <-- YOUR ANSWER HERE
    bondprice = np.where(default_time>mat, np.exp(-rf_rate*mat),recovery*np.exp(-rf_rate*default_time)) ### <-- YOUR ANSWER HERE
    return np.mean(bondprice)

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
# Find the mean and std by calling the function 100 times. 
range100 = [corp_bond() for i in range(100)] ### <-- YOUR ANSWER HERE
meanbond = np.mean(range100) ### <-- YOUR ANSWER HERE
stdbond = np.std(range100) ### <-- YOUR ANSWER HERE

print(meanbond)
print(stdbond)

0.561302755822034
0.0018521126849184583


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [6]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size= n_sample//2)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.000000000000002, 10.0)

In [14]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=10, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    ### <--
    if(antithetic): 
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else: default_time = -(1/def_rate)*np.log(U)
    if(mean_match): 
        default_time += 1/def_rate-default_time.mean()
    price= np.where(default_time>mat, np.exp(-rf_rate*mat),recovery*np.exp(-rf_rate*default_time)) ### <-- YOUR ANSWER HERE
    return np.mean(price)

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
# For (i) Antithetic:
anti = [corp_bond_cv(antithetic=True, mean_match=False) for i in range(100)]
print(np.mean(anti), np.std(anti))
# For (ii) Mean matching: 
mm = [corp_bond_cv(antithetic=False, mean_match=True) for i in range(100)]
print(np.mean(mm), np.std(mm))
#For both: 
both = [corp_bond_cv(antithetic=True, mean_match=True) for i in range(100)]
print(np.mean(both), np.std(both))

0.5589928051404138 0.0
0.5561465865223204 1.1102230246251565e-16
0.5561171593210134 1.1102230246251565e-16


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [80]:
### Put the analytic expression for the corporate bond price 

#The expression is obtained by: If default (pdf) + Not default (cdf)

def corp_bondan(def_rate = 0.03, rf_rate = 0.04, recovery = 0.3, mat = 10): 
    bondpricean = (recovery*def_rate*(1-np.exp((-rf_rate-def_rate)*mat)))/(rf_rate+def_rate)+np.exp((-rf_rate-def_rate)*mat) 
    return np.mean(bondpricean) 
corp_bondan() ### <-- YOUR ANSWER HERE

0.5613100504467997